<img src="../../images/qiskit-heading.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px" align="left">

# Jupyter Tools for Terra

In this section, we will learn how to monitor the status of jobs submitted to devices and simulators (collectively called backends), as well as discover how to easily query backend details and view the collective state of all the backends available to you.

## Loading the Monitoring Tools

First, let us load the default qiskit routines, and register our IBMQ credentials.

In [1]:
from qiskit import *
IBMQ.load_accounts()

Functions for monitoring jobs and backends are here:

In [2]:
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

If you are running inside a Jupyter notebook, then you will also need to have `ipywidgets` version `7.3.0` or higher installed.  These come pre-installed in Anaconda.  There are also Jupyter notebook 'magics' available for each of the abover functions.  The following will register those magics, making them ready for use.

In [3]:
from qiskit.tools.jupyter import *

## Tracking Job Status

Many times a job(s) submitted to the IBM Q network can take a long time to process, e.g. jobs with many circuits and/or shots, or may have to wait in queue for other users.  In situations such as these, it is beneficial to have a way of monitoring the progress of a job, or several jobs at once.  As of Qiskit `0.6+` it is possible to monitor the status of a job in a Jupyter notebook, and also in a Python script (verision `0.7+`).

Lets see how to make use of these tools.

### Monitoring the status of a single job

Lets build a simple Bell circuit, submit it to a device, and then monitor its status.

In [4]:
q = QuantumRegister(2)
c = ClassicalRegister(2)
qc = QuantumCircuit(q, c)

qc.h(q[0])
qc.cx(q[0], q[1])
qc.measure(q, c);

Lets grab the least busy backend

In [6]:
from qiskit.providers.ibmq import least_busy
backend = least_busy(IBMQ.backends(simulator=False))
backend.name()

'ibmqx4'

Monitor the job using `job_monitor` in blocking-mode (i.e. using the same thread as the Python interpretor)

In [7]:
job1 = execute(qc, backend)
job_monitor(job1)

HTML(value="<p style='font-size:16px;'>Job Status: job is being initialized </p>")

Monitor the job using `job_monitor` in async-mode (Jupyter notebooks only).  The job will be monitored in a separate thread, allowing you to continue to work in the notebook.

In [8]:
job2 = execute(qc, backend)
job_monitor(job2, monitor_async=True)

HTML(value="<p style='font-size:16px;'>Job Status: job is being initialized </p>")

It is also possible to monitor the job using the `qiskit_job_status` Jupyter notebook magic.  This method is always asyncronous.

In [9]:
%%qiskit_job_status
job3 = execute(qc, backend)

Note that, for the `qiskit_job_status` to work, the job returned by `execute` must be stored in a variable so that it may be retrieved by the magic.

### Monitoring many jobs simultaneously

Here we will monitor many jobs sent the the device.  It is if the jobs are stored in a list to make retrevial easier.

In [10]:
num_jobs = 5
my_jobs = []
for j in range(num_jobs):
    my_jobs.append(execute(qc, backend))
    job_monitor(my_jobs[j], monitor_async=True)

HTML(value="<p style='font-size:16px;'>Job Status: job is being initialized </p>")

HTML(value="<p style='font-size:16px;'>Job Status: job is being initialized </p>")

HTML(value="<p style='font-size:16px;'>Job Status: job is being initialized </p>")

HTML(value="<p style='font-size:16px;'>Job Status: job is being initialized </p>")

HTML(value="<p style='font-size:16px;'>Job Status: job is being initialized </p>")

Or, using magic:

In [11]:
%%qiskit_job_status
my_jobs2 = []
for j in range(num_jobs):
    my_jobs2.append(execute(qc, backend))

In the magics example, the magic is smart enough to know that the list `my_jobs2` contains jobs, and will automatically extract them and check their status.  We are not limited to using `jobs.append()`, and can use an indexed list or NumPy array as well:

In [12]:
%%qiskit_job_status
import numpy as np
my_jobs3 = np.empty(num_jobs, dtype=object)
for j in range(num_jobs):
    my_jobs3[j] = execute(qc, backend)

### Changing the interval of status updating

By default, the interval at which the job status is checked is every two seconds.  However, the user is free to change this using the `interval` keyword argument in `job_monitor`

In [13]:
job3 = execute(qc, backend)
job_monitor(job3, interval=5)

HTML(value="<p style='font-size:16px;'>Job Status: job is being initialized </p>")

and the `-i` or `--interval` arguments to the Jupyter magic.

In [14]:
%%qiskit_job_status -i 5
job4 = execute(qc, backend)

In [15]:
%%qiskit_job_status --interval 5
job5 = execute(qc, backend)

## Backend Details

So far we have been executing our jobs on a backend, but we have explored the backends in any detail.  For example, we have found the least busy backend, but do not know if this is the best backend with respect to gate errors, topology etc.  It is possible to get detailed information for a single backend by calling `backend_monitor`:

In [16]:
backend_monitor(backend)

ibmqx4
Configuration
-------------
    n_qubits: 5
    operational: True
    status_msg: active
    pending_jobs: 4
    basis_gates: ['u1', 'u2', 'u3', 'cx', 'id']
    local: False
    simulator: False
    open_pulse: False
    credits_required: True
    conditional: False
    max_experiments: 75
    max_shots: 8192
    coupling_map: [[1, 0], [2, 0], [2, 1], [3, 2], [3, 4], [4, 2]]
    sample_name: raven
    description: 5 qubit device
    n_registers: 1
    memory: True
    url: None
    backend_version: 1.0.0
    backend_name: ibmqx4
    online_date: 2018-11-06T05:00:00+00:00
    allow_q_object: True

Qubits [Name / Freq / T1 / T2 / U1 err / U2 err / U3 err / Readout err]
-----------------------------------------------------------------------
    Q0 / 5.24985 GHz / 52.87796 µs / 45.91462 µs / 0.0 / 0.00077 / 0.00155 / 0.06025
    Q1 / 5.29578 GHz / 52.18911 µs / 19.45196 µs / 0.0 / 0.00197 / 0.00395 / 0.07425
    Q2 / 5.35325 GHz / 42.88025 µs / 29.48086 µs / 0.0 / 0.00163 / 0.00326 

Or, if we are interested in a higher-level view of all the backends available to us, then we can use `backend_overview()`

In [17]:
backend_overview()

ibmq_20_tokyo               ibmq_16_melbourne            ibmqx4
-------------               -----------------            ------
Num. Qubits:  20            Num. Qubits:  14             Num. Qubits:  5
Pending Jobs: 0             Pending Jobs: 3              Pending Jobs: 6
Least busy:   True          Least busy:   False          Least busy:   False
Operational:  True          Operational:  True           Operational:  True
Avg. T1:      86.9          Avg. T1:      50.3           Avg. T1:      47.2
Avg. T2:      55.3          Avg. T2:      63.0           Avg. T2:      24.7





There are also Jupyter magic equivalents that give more detailed information.

In [18]:
%qiskit_backend_monitor backend

/Users/jaygambetta/gitshare/qiskit-terra/qiskit/providers/ibmq/ibmqbackend.py:180: DeprecationWarning: Some jobs (281) are in a no-longer supported format. These jobs will stop working after Qiskit 0.7. Save the results or send the job with Qiskit 0.7+. Old jobs:
 - 5c156b801d475200611467ee
 - 5c156b2a737f7600571ec66a
 - 5c156a49557dff00576be243
 - 5c15694eee03b40054b35f45
 - 5c156921f07ad5005c5df729
 - 5c156877a57edb00531278e1
 - 5c156851ed804c00561942b7
 - 5c15682513eb0c005321c326
 - 5c156762d38acd00557c9bda
 - 5c15672a737f7600571ec65e
 - 5c1565fded804c00561942ac
 - 5c1564ccaa0f280056346f61
 - 5c15559ba57edb00531278ae
 - 5c0ac161751a790052faca8e
 - 5c0ac01d1d0f11005de5d7c1
 - 5c0abee6ab6cf6005c8d312c
 - 5bfe0ec5c9630c0055d26435
 - 5bfe0e0f3626d5005e226360
 - 5be8f55b28716d00547b70ad
 - 5be8f54e9a9893006ff69200
 - 5be8f39c54dacb0059c2b0db
 - 5be8f394afd471005540d4d7
 - 5be8ae5e17436b0052751909
 - 5be748a7e00f60005ad7f23d
 - 5be746e3d4d36f0054595d60
 - 5bc3e88d404ceb006174af14
 - 5bc3e

The Jupyter `backend_overview` runs live in the notebook, and will automatically update itself every minute.

In [19]:
%qiskit_backend_overview